In [ ]:
import pandas as pd
import numpy as np
from imblearn.over_sampling import SMOTE
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import matplotlib.pyplot as plt

# 定义要处理的主题
train_subjects = ['Carpenter1', 'Carpenter2', 'Rebar1', 'Rebar2', 'Rebar3']
test_subjects = ['Masonry1', 'Masonry2']

def load_and_preprocess_data(subject, label_mapping):
    """
    加载并预处理指定主题的数据。
    
    参数:
        subject (str): 主题名称
        label_mapping (dict): 标签重编码映射
    
    返回:
        tuple: 处理后的特征X和标签y
    """
    # 读取数据
    df = pd.read_csv(f'../{subject}.csv', low_memory=False)
    
    # 清除空值行
    df_cleaned = df.dropna()
    
    # 转换 'Label_2' 列为数值类型
    df_cleaned.loc[:, 'Label_2'] = pd.to_numeric(df_cleaned['Label_2'], errors='coerce')
    
    # 只选取前八个动作
    df_cleaned = df_cleaned[df_cleaned['Label_2'] <= 8]
    
    # 重新编码标签
    df_cleaned.loc[:, 'Label_2'] = df_cleaned['Label_2'].map(label_mapping)

    # 同步 xd 和 kd 的记录
    df_xd = df_cleaned[df_cleaned['position'] == 'xd'].copy()
    df_kd = df_cleaned[df_cleaned['position'] == 'kd'].copy()

    row_len = min(len(df_xd), len(df_kd))

    # 获取列名并添加后缀以避免重复
    columns_xd = [f'{col}_xd' for col in df_xd.columns]
    columns_kd = [f'{col}_kd' for col in df_kd.columns]

    # 重命名列
    df_xd.columns = columns_xd
    df_kd.columns = columns_kd

    df_xd = df_xd.iloc[:row_len-1, :]
    df_kd = df_kd.iloc[:row_len-1, :]

    df_xd = df_xd.reset_index(drop=True)
    df_kd = df_kd.reset_index(drop=True)

    # 合并两个DataFrame
    paired_df = pd.concat([df_kd, df_xd], axis=1)
    paired_df = paired_df.loc[:, ~paired_df.columns.duplicated()]

    # 删除不需要的列
    columns_to_drop = ['seconds_elapsed_xd','seconds_elapsed_kd', 'position_xd','position_kd', 'time_xd', 'time_kd', 'Real Time_xd', 'Real Time_kd']
    paired_df = paired_df.drop(columns=columns_to_drop)
    
    X = paired_df.drop(['Label_2_xd', 'Label_2_kd', 'Subject_kd', 'Subject_xd'], axis=1)
    y = paired_df['Label_2_xd']

    print(X.columns)

    # 确保所有特征都是数值类型
    for i in X.columns:
        if X[i].dtype == 'object':
            X[i] = pd.to_numeric(X[i], errors='raise')

    # 设置窗口大小并应用滑动窗口聚合
    window_size = 30
    X_rolled = X.rolling(window=window_size, min_periods=1).mean()
    y_rolled = y.rolling(window=window_size, min_periods=1).apply(
        lambda x: x.mode().iloc[0] if not x.isnull().all() else None, raw=False
    )

    # 删除没有完整窗口的数据
    X_rolled = X_rolled.dropna().reset_index(drop=True)
    y_rolled = y_rolled.dropna().reset_index(drop=True)

    # 使用SMOTE处理类别不平衡
    smote = SMOTE(random_state=42)
    X_final, y_final = smote.fit_resample(X_rolled, y_rolled)

    # 将y标签修改成int
    y_final = y_final.astype(int) - 1

    return X_final, y_final

def prepare_datasets(train_subjects, test_subjects, label_mapping):
    """
    准备训练集和测试集。
    
    参数:
        train_subjects (list of str): 训练集主题列表
        test_subjects (list of str): 测试集主题列表
        label_mapping (dict): 标签重编码映射
    
    返回:
        tuple: 训练集特征、训练集标签、测试集特征、测试集标签
    """
    all_train_data = []
    all_test_data = []

    for subject in train_subjects + test_subjects:
        X, y = load_and_preprocess_data(subject, label_mapping)
        if subject in train_subjects:
            all_train_data.append((X, y))
        else:
            all_test_data.append((X, y))

    # 合并所有训练集和测试集数据
    X_train = pd.concat([x for x, _ in all_train_data], axis=0).values
    y_train = pd.concat([y for _, y in all_train_data], axis=0).values
    X_test = pd.concat([x for x, _ in all_test_data], axis=0).values
    y_test = pd.concat([y for _, y in all_test_data], axis=0).values

    return X_train, y_train, X_test, y_test

def plot_confusion_matrix(y_true, y_pred, labels, title, save_path):
    """
    绘制混淆矩阵图。
    
    参数:
        y_true (array-like): 真实标签
        y_pred (array-like): 预测标签
        labels (list of str): 分类标签
        title (str): 图标题
        save_path (str): 保存图片路径
    """
    cm = confusion_matrix(y_true, y_pred)
    fig, ax = plt.subplots()
    im = ax.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
    ax.figure.colorbar(im, ax=ax)
    ax.set(xticks=np.arange(cm.shape[1]),
           yticks=np.arange(cm.shape[0]),
           xticklabels=labels, yticklabels=labels,
           title=title,
           ylabel='True label',
           xlabel='Predicted label')
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(j, i, format(cm[i, j], 'd'),
                    ha="center", va="center",
                    color="white" if cm[i, j] > thresh else "black")
    fig.tight_layout()
    plt.savefig(save_path, dpi=300)

    # 标签重编码映射
label_mapping = {1: 1, 2: 2, 3: 3, 4: 3, 5: 4, 6: 3, 7: 3, 8: 5}

# 准备数据集
X_train, y_train, X_test, y_test = prepare_datasets(train_subjects, test_subjects, label_mapping)

Index(['Accelerometer.xlsx_z_kd', 'Accelerometer.xlsx_y_kd',
       'Accelerometer.xlsx_x_kd', 'Gravity_z_kd', 'Gravity_y_kd',
       'Gravity_x_kd', 'Gyroscope_z_kd', 'Gyroscope_y_kd', 'Gyroscope_x_kd',
       'Magnetometer_z_kd', 'Magnetometer_y_kd', 'Magnetometer_x_kd', 'yaw_kd',
       'qx_kd', 'qz_kd', 'roll_kd', 'qw_kd', 'qy_kd', 'pitch_kd',
       'Accelerometer.xlsx_z_xd', 'Accelerometer.xlsx_y_xd',
       'Accelerometer.xlsx_x_xd', 'Gravity_z_xd', 'Gravity_y_xd',
       'Gravity_x_xd', 'Gyroscope_z_xd', 'Gyroscope_y_xd', 'Gyroscope_x_xd',
       'Magnetometer_z_xd', 'Magnetometer_y_xd', 'Magnetometer_x_xd', 'yaw_xd',
       'qx_xd', 'qz_xd', 'roll_xd', 'qw_xd', 'qy_xd', 'pitch_xd'],
      dtype='object')
Index(['Accelerometer.xlsx_z_kd', 'Accelerometer.xlsx_y_kd',
       'Accelerometer.xlsx_x_kd', 'Gravity_z_kd', 'Gravity_y_kd',
       'Gravity_x_kd', 'Gyroscope_z_kd', 'Gyroscope_y_kd', 'Gyroscope_x_kd',
       'Magnetometer_z_kd', 'Magnetometer_y_kd', 'Magnetometer_x_kd', 'ya